# CMS3504-7504 Big Data Analysis - Assignment 2

In [1]:
import findspark
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
findspark.init(r"C:\Users\rejee\spark-3.3.1-bin-hadoop3")

In [3]:
import pyspark

In [4]:
spark = SparkSession.builder.appName("AmazonKindle").getOrCreate()

In [5]:
sc = spark.sparkContext

In [6]:
#reading data
df = spark.read.load(".\kindle_reviews.csv", format="csv", sep=",", header="true", inferSchema=True)
df.show(10)

+---+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|_c0|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+---+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|  0|B000F83SZQ| [0, 0]|      5|I enjoy vintage b...| 05 5, 2014|A1F6404F1VG29J|          Avidreader|  Nice vintage story|    1399248000|
|  1|B000F83SZQ| [2, 2]|      4|This book is a re...| 01 6, 2014| AN0N05A9LIJEQ|            critters|        Different...|    1388966400|
|  2|B000F83SZQ| [2, 2]|      4|This was a fairly...| 04 4, 2014| A795DMNCJILA6|                 dot|               Oldie|    1396569600|
|  3|B000F83SZQ| [1, 1]|      5|I'd never read an...|02 19, 2014|A1FV0SX13TWVXQ|"Elaine H. Turley...|  I really liked it.|    1392768000|
|  4|B000F83SZQ| [0, 1]|      4|If

In [7]:
#printing schema to know the co
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- helpful: string (nullable = true)
 |-- overall: integer (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: string (nullable = true)



In [8]:
#Descriptive Analysi
df.describe().show()

+-------+----------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+
|summary|             _c0|                asin|             helpful|           overall|          reviewText|          reviewTime|          reviewerID|reviewerName|             summary|      unixReviewTime|
+-------+----------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+
|  count|          982820|              982815|              982815|            982619|              982597|              982605|              982598|      978957|              982385|              982397|
|   mean|        491309.0|                null|1.3761077877551022E9| 4.347801131466011|                null|   270.1666666666667|   488.4230769230769|         NaN|            I

In [9]:
#Descriptive Analysis of overall
kindledf.select("overall").describe().show()

NameError: name 'kindledf' is not defined

In [ ]:
from pyspark.sql.functions import count, sum

# Count the number of reviews per score
review_counts = df.groupBy("overall").agg(count("*").alias("count"))

# Display the results
review_counts.show()

In [ ]:
pandas_df = review_counts.toPandas()

# Create a bar chart
pandas_df.plot(x='overall', y='count', kind='bar', figsize=(10,6), color=['g', 'r', 'b', 'm', 'y'])
plt.grid()
plt.xlabel("overall")
plt.ylabel("Number of Reviews Per overall")
plt.title("Distribution of Reviews Per overall")

# Show the plot
plt.show()

In [ ]:
#Descriptive Analysis of reviewername
kindledf.select("reviewerName").describe().show()

In [ ]:
kindledf.na.drop().show()

In [ ]:
#Number of rows and columns
print((kindledf.count(),len(kindledf.columns)))

In [ ]:
print(kindledf.count())

In [ ]:
#Distribution of reviews per score (value counts)

kindledf = df.groupBy('overall').count()
kindledf

In [ ]:
kindledf_overall.show()

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
dafr = kindledf_overall.toPandas()
plt.figure(figsize=(8,7))
sns.barplot(x='count', y='overall', data=dafr)
plt.title("Overall scores and the Count of overall scores")
plt.xlim(10000,600000)
plt.show()

In [ ]:
import pandas as pd
overall_data = dafr["overall"]
count_data = dafr["count"]
fig=plt.figure(figsize=(5,5))
fig.patch.set_facecolor('w')
explode = (0.1,0,0,0,0)
#Plot
plt.pie(count_data,autopct='%1.1f%%',startangle=90,explode=explode)
plt.title('Overall scores and their counts',bbox={'facecolor':'0.8','pad':4})
plt.legend(overall_data,loc="lower left")

In [ ]:
#Creating a dataframe with the columns that will be mainly required
amazondf = kindledf.select('overall','reviewText')

In [ ]:
amazondf.show(5)

In [ ]:
amazondf.count()

### Feature Extraction

In [ ]:
#Loading packages
import pyspark.ml.feature

In [ ]:
#Loading transformer and extractor packages
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF
from pyspark.ml.feature import StringIndexer

In [ ]:
#Stages for Pipeline
tokenizer = Tokenizer(inputCol='reviewText',outputCol='mytokens')
stopwords_remover = StopWordsRemover(inputCol='mytokens',outputCol='filterd_tokens')
vectorizer = CountVectorizer(inputCol='filtered_tokens',outputCol='rawFeatures')
idf = IDF(inputCol='rawFeatures',outputCol='vectorizedFeatures')

In [ ]:
#Score indexing
scoreIndexer = StringIndexer(inputCol='overall',outputCol='score').fit(df)
scoreIndexer.transform(df).show(5)

In [ ]:
scoreIndexer.labels

In [ ]:
#Dictionary of labels
label_dict = {'1':0.0,'2':1.0,'3':2.0,'4':3.0,'5':4.0}

In [ ]:
amazondf.show(5)

In [ ]:
#amazondf = scoreIndexer.transform(amazondf).show(5)

In [ ]:
#amazondf.na.drop().show()

In [ ]:
# Splitting dataset
(trainDF,testDF) = df.randomSplit((0.7,0.3),seed=42)